In [1]:
import pandas as pd
import numpy as np

In [2]:
loc_lbl = r"D:\FacundoTorraca\Documents\TP2_Machine_Learning\Labels"

In [56]:
loc_ftr = r"D:\FacundoTorraca\Documents\TP2_Machine_Learning\Features\ftr_ins"

In [3]:
ins_lb = pd.read_csv( loc_lbl + "\\ins_lb.csv", dtype = {"obs_18_20":"bool", "obs_21_23":"bool", "obs_24_26":"bool"} )

### Filtro de manera que tenga los datos necesarios

In [154]:
rh_18_20 = pd.read_csv( "D:\FacundoTorraca\Documents\TP2_Machine_Learning\Training Sets\ins_18_20.csv" ); rh_18_20 = rh_18_20["ref_hash"].drop_duplicates().tolist()
rh_21_23 = pd.read_csv( "D:\FacundoTorraca\Documents\TP2_Machine_Learning\Training Sets\ins_21_23.csv" ); rh_21_23 = rh_21_23["ref_hash"].drop_duplicates().tolist()

In [155]:
dt_18_20 = ins_lb.loc[ ins_lb["ref_hash"].isin( rh_18_20 ), ["ref_hash"] ] #Datos del (18-20)
sc_21_23 = ins_lb.loc[ ins_lb["ref_hash"].isin( rh_18_20 ), ["21_23_sc"] ] #SC del (21-23)

dt_21_23 = ins_lb.loc[ ins_lb["ref_hash"].isin( rh_21_23 ), ["ref_hash"] ] #Datos del (21-23)
sc_24_26 = ins_lb.loc[ ins_lb["ref_hash"].isin( rh_21_23 ), ["24_26_sc"] ] #SC del (24-26)

# <span style="color:orange">  Preparo los datos para predecir </span>
Con los datos 21-23 predecimos "24-26_sc"

In [257]:
Z = dt_21_23

### Agregamos los features 

In [258]:
ftr_1_Z = pd.read_csv( loc_ftr + "\\sin_hour_21_23.csv" ) 
ftr_2_Z = pd.read_csv( loc_ftr + "\\cant_app_ins_21_23.csv")
ftr_3_Z = pd.read_csv( loc_ftr + "\\cant_imp_21_23.csv" )

In [259]:
Z = Z.merge( ftr_1_Z, how = "inner", on = "ref_hash" )
Z = Z.merge( ftr_2_Z, how = "inner", on = "ref_hash" )
Z = Z.merge( ftr_3_Z, how = "inner", on = "ref_hash" )

# <span style="color:orange">  Preparo los datos para entrenar </span>

Con los datos 18-20 + "21-23_sc" entrenamos

In [264]:
Y = sc_21_23

In [265]:
X = dt_18_20

### Agregamos los features

In [266]:
ftr_1_X = pd.read_csv( loc_ftr + "\\sin_hour_18_20.csv" )
ftr_2_X = pd.read_csv( loc_ftr + "\\cant_app_ins_18_20.csv")
ftr_3_X = pd.read_csv( loc_ftr + "\\cant_imp_18_20.csv" )

In [267]:
X = X.merge( ftr_1_X, how = "inner", on = "ref_hash" )
X = X.merge( ftr_2_X, how = "inner", on = "ref_hash" )
X = X.merge( ftr_3_X, how = "inner", on = "ref_hash" )

# Prediccion con <span style="color:green"> *XGBoost*</span> 

In [269]:
import xgboost as xgb

In [270]:
estimator = xgb.XGBRegressor(objective ='reg:linear', 
                             colsample_bytree = 0.3, 
                             learning_rate = 0.1,
                             max_depth = 5, 
                             alpha = 10, 
                             n_estimators = 100)

In [271]:
pmt = estimator.fit( X,Y )

In [272]:
prd = estimator.predict( Z )

### Calculamos el RMSE

In [273]:
from sklearn.metrics import mean_squared_error

In [274]:
rmse = np.sqrt( mean_squared_error(ans, prd) )
print("RMSE: %f" % (rmse))

RMSE: 25552.230611
